# Deploy a model to Modzy and run it all from inside a Jupyter notebook!

No need for Docker here!

In [1]:
import sklearn
import modzymodel

### Define required variables

In [2]:
user_api_key = 'XXXX'

input_file = '../../builder_api/test/sklearn/data/0.jpg'

### Use the SDK to interact with Modzy

#### Build the image and upload it to Modzy

For every input there must be one output of type json. The "Output Name" of every output must be the same as the "Model Input Filename" of the referenced input but the extension ".json" must be added (even if it's already a json).
E.g.: if the "Model Input Filename" of one input is "digit.jpg", the "Output Name" of its corresponding output should be "digit.jpg.json".

Job input names must match this name.

In [3]:
image_data = {
    'name': 'sdk-test-sklearn',
    'version': '0.0.1',
    'model_name': 'digits',
    'model_path': '../../builder_api/test/sklearn/example_model',
}

metadata = {
    'inputs': [{
        'acceptedMediaTypes': 'image/jpeg',
        'description': 'description input',
        'maximumSize': int(1.024e6),
        'name': 'digit.jpg'
    }],
    'outputs': [{
        'description': 'description output',
        'maximumSize': int(1.024e6),
        'mediaType': 'application/json',
        'name': 'digit.jpg.json'
    }],
}

Finally, we are able to publish that image to Modzy in order to create our draft model.

In [4]:
res = modzymodel.publish(
    api_key=user_api_key,
    module=sklearn,
    image_data=image_data,
    deploy=True,
    input_example_path=input_file,
    metadata=metadata
)

error = res.get('error')
job_id = res.get('job_id')

if error:
    print('Error:', error)
else:
    print('Job ID:', job_id)

Publishing container... Ok!
Job ID: modzy-builder-job-212de1db-440f-44bb-a2af-8442100d44f0


In [5]:
model_data = modzymodel.get_job_status(job_id)
model_data = model_data.get('result')

Deployed model: https://master.dev.modzy.engineering/models/khwkpvuhrq-modzy-sdk-test-sklearn/0.0.1


# Inference!

Retrieve model data and use it to run a job. Take into account that input names must match model input filenames.

In [7]:
input_name = metadata['inputs'][0]['name'] # e.g. digit.jpg

request_data = {
  'model': {
    'identifier': f'{model_data.get("identifier")}',
    'version': f'{model_data.get("version")}'
  },
  'input': {
    'type': 'embedded',
    'sources': {
      'image-1': {
        input_name: 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/wAALCAAIAAgBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APwRha4fw9NG6sqgqVZ1+Vxnovvn61//2Q=='
        # ^ this has to match the input filename specified when the model was created
      }
    }
  }
}

import requests

res_job = requests.post(
    'https://master.dev.modzy.engineering/api/jobs',
    json=request_data,
    headers={'Authorization': f'ApiKey {user_api_key}'}
)

res_job_json = res_job.json()

res_job_json

{'model': {'identifier': 'khwkpvuhrq', 'version': '0.0.1'},
 'totalInputs': 1,
 'jobIdentifier': '6c32f736-a578-4340-9a0b-9eb92d38d122',
 'accessKey': 'LzYMlkVfj5780fKHQzrK',
 'explain': False,
 'jobInputs': {'identifier': ['image-1']},
 'inputByteAmount': 553,
 'submittedAt': '2021-03-26T13:55:35.204+00:00'}

Wait until the job has finished and retrieve the results.

In [10]:
job_id = res_job_json.get('jobIdentifier')

res_result = requests.get(
    f'https://master.dev.modzy.engineering/api/results/{job_id}',
    headers={'Authorization': f'ApiKey {user_api_key}'}
)

res_result_json = res_result.json()

res_result_json

{'jobIdentifier': '6c32f736-a578-4340-9a0b-9eb92d38d122',
 'total': 1,
 'completed': 0,
 'failed': 0,
 'finished': False,
 'submittedByKey': 'LzYMlkVfj5780fKHQzrK',
 'explained': False}